In [1]:
# import os

# # 设置环境变量
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# # 打印环境变量以确认设置成功
# print(os.environ.get('HF_ENDPOINT'))

# import subprocess
# import os

# result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
# output = result.stdout
# for line in output.splitlines():
#     if '=' in line:
#         var, value = line.split('=', 1)
#         os.environ[var] = value

In [2]:
import json
import random
import re
import os
from datasets import load_dataset
from openai import OpenAI
from sklearn.metrics import accuracy_score, classification_report

In [3]:
# ==========================================
# 1. 配置 API  https://bailian.console.aliyun.com/
# ==========================================
API_KEY = "sk-"  # 注意：在生产环境中请勿硬编码，建议使用 os.getenv
MODEL_ID = "qwen3-max"

client = OpenAI(
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=API_KEY,
)

In [4]:
# ==========================================
# 2. 数据准备 (加载、采样、打乱) - TF Binding 版本
# ==========================================
print("Loading dataset...")
try:
    # 假设你的本地数据集路径
    dataset = load_dataset('dnagpt/dna_transcription_factor_prediction')
    # 获取训练集
    ds = dataset['train']
    
    # ⚠️ 关键修改：检查数据集的列名
    # 通常列名可能是 'sequence', 'seq', 'dna', 'sentence' 等
    # 这里假设列名就是你描述的 'sequence'
    print(f"Dataset columns: {ds.column_names}") 
    
    # 1. 分离数据 (Label 0 和 Label 1)
    # 注意：确保这里使用的是正确的键名，这里假设是 'label' 和 'sequence'
    data_label_0 = [item for item in ds if item['label'] == 0]
    data_label_1 = [item for item in ds if item['label'] == 1]
    
    # 2. 随机采样 (各50个，凑100个测试)
    random.seed(42)
    sample_num = 80
    
    # 防止数据不够报错，加个 min 取值
    real_sample_num_0 = min(sample_num, len(data_label_0))
    real_sample_num_1 = min(sample_num, len(data_label_1))
    
    sampled_0 = random.sample(data_label_0, real_sample_num_0)
    sampled_1 = random.sample(data_label_1, real_sample_num_1)
    
    # 3. 合并并打乱
    combined_data = sampled_0 + sampled_1
    random.shuffle(combined_data)
    
    print(f"Data prepared: {len(combined_data)} sequences (Positive: {len(sampled_1)}, Negative: {len(sampled_0)})")

except Exception as e:
    print(f"Error loading dataset: {e}")
    # 打印详细错误方便调试
    import traceback
    traceback.print_exc()
    exit()

# ==========================================
# 构建用于 Prompt 的 JSON List
# ==========================================
prompt_data_list = []
id_to_ground_truth = {}

for idx, item in enumerate(combined_data, 1):
    # ⚠️ 关键修改：单序列任务，JSON 里不需要 seq_a, seq_b
    # 只需要一个 "sequence" 字段
    
    # 容错：有些数据集把序列叫 'sentence'，有些叫 'sequence'，有些叫 'seq'
    # 这里的逻辑是优先找 'sequence'，找不到就找 'sentence'
    seq_content = item.get('sequence', item.get('sentence', item.get('seq', '')))
    
    prompt_data_list.append({
        "id": idx,
        "sequence": seq_content # 👈 只要这一条
    })
    
    # 记录真值
    id_to_ground_truth[idx] = item['label']

print("Prompt data constructed.")

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset './dna_transcription_factor_prediction' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


Loading dataset...
Dataset columns: ['sequence', 'label']
Data prepared: 160 sequences (Positive: 80, Negative: 80)
Prompt data constructed.


In [5]:
# ==========================================
# 3. 构建 Prompt (纯抽象分类 / 无偏见版)
# ==========================================

# System Prompt: 设定为一个冷酷无情的模式识别机器
# 不提 Signal, Noise, Sentiment, Biology
system_prompt = """You are an advanced Pattern Recognition Engine specialized in analyzing abstract sequence data.

1. The Task:
You are provided with a dataset containing sequences from two distinct latent categories (Category 0 and Category 1).
These two categories follow different underlying generative rules or statistical distributions.

2. Your Objective:
Analyze the structural consistency of the sequences to separate them into two groups.
* **Group A**: Sequences that share a common set of latent features (e.g., specific sub-patterns, repetition rates, or complexity profiles).
* **Group B**: Sequences that share a *different* set of features, or lack the features found in Group A.

3. Output Rules:
* Assign **Label 1** to one distinct group.
* Assign **Label 0** to the other distinct group.
* The assignment is arbitrary, but the grouping must be consistent based on the patterns you detect.
* Return a RAW JSON object: `[{"id": 1, "prediction": 1}, {"id": 2, "prediction": 0}, ...]`
"""

# User Prompt: 仅要求“区分差异”
user_prompt = f"""Here is a batch of raw sequences.
The data is a mixture of two hidden classes.

Task: Perform binary classification based on internal sequence similarity.
* Identify the two dominant patterns/styles in the data.
* Classify the sequences into **0** or **1** based on which pattern they best fit.

Data:
{json.dumps(prompt_data_list, indent=2)}
"""

In [6]:
# ==========================================
# 4. 调用 Volcengine API
# ==========================================
print("-" * 30)
print(f"Calling Volcengine Model: {MODEL_ID}...")

try:
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.1, # 低温以保证输出格式稳定
        top_p=0.9,
    )
    
    full_content = response.choices[0].message.content.strip()
    print("Response received.")
    # 打印前200个字符用于调试
    print(f"Response snippet: {full_content[:200]}...")

except Exception as e:
    print(f"API Call Failed: {e}")
    full_content = ""

------------------------------
Calling Volcengine Model: qwen3-max...
Response received.
Response snippet: ```json
[{"id": 1, "prediction": 1}, {"id": 2, "prediction": 1}, {"id": 3, "prediction": 0}, {"id": 4, "prediction": 1}, {"id": 5, "prediction": 1}, {"id": 6, "prediction": 1}, {"id": 7, "prediction":...


In [7]:
# ==========================================
# 5. 解析结果与评估 (自动处理标签反转版)
# ==========================================
import re
import json
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def parse_llm_json(text):
    """提取并解析 JSON"""
    try:
        code_block = re.search(r"```json\s*(\[.*?\])\s*```", text, re.DOTALL)
        if code_block:
            return json.loads(code_block.group(1))
        match = re.search(r"\[.*\]", text, re.DOTALL)
        if match:
            return json.loads(match.group(0))
        return json.loads(text)
    except:
        return []

# 解析
predictions_list = parse_llm_json(full_content)

y_true = []
y_pred_raw = [] # 原始预测

print("-" * 30)
if not predictions_list:
    print("Failed to parse JSON.")
else:
    print(f"Parsed {len(predictions_list)} predictions.")
    
    for item in predictions_list:
        p_id = item.get('id')
        p_val = item.get('prediction')
        
        if p_id in id_to_ground_truth and p_val in [0, 1]:
            y_true.append(id_to_ground_truth[p_id])
            y_pred_raw.append(int(p_val))

    if y_true:
        # 1. 计算原始准确率
        acc_raw = accuracy_score(y_true, y_pred_raw)
        print(f"\n[Original] Accuracy: {acc_raw:.2%}")
        
        # 2. 检查是否需要反转 (Anti-correlation Check)
        final_y_pred = y_pred_raw
        is_flipped = False
        
        if acc_raw < 0.5:
            print("\n⚠️ Detected Label Flipping (Accuracy < 50%)!")
            print("The model found the pattern but swapped the labels.")
            print("Inverting predictions (0->1, 1->0)...")
            
            # 执行反转：0变1，1变0
            final_y_pred = [1 - y for y in y_pred_raw]
            is_flipped = True
        
        # 3. 计算最终指标
        acc_final = accuracy_score(y_true, final_y_pred)
        print(f"\n[Corrected] Final Accuracy: {acc_final:.2%}")
        
        print("\nClassification Report (After Correction):")
        # 这里的 target_names 顺序固定是 [0, 1]
        print(classification_report(y_true, final_y_pred, target_names=["Unbound (0)", "Bound (1)"]))
        
        # 打印混淆矩阵看一眼
        print("\nConfusion Matrix:")
        print(confusion_matrix(y_true, final_y_pred))

        result_log = {
            "accuracy": acc_final,
            "flipped": is_flipped,
            "predictions": predictions_list
        }
    else:
        print("No valid matching IDs found.")

------------------------------
Parsed 160 predictions.

[Original] Accuracy: 62.50%

[Corrected] Final Accuracy: 62.50%

Classification Report (After Correction):
              precision    recall  f1-score   support

 Unbound (0)       0.69      0.45      0.55        80
   Bound (1)       0.59      0.80      0.68        80

    accuracy                           0.62       160
   macro avg       0.64      0.62      0.61       160
weighted avg       0.64      0.62      0.61       160


Confusion Matrix:
[[36 44]
 [16 64]]
